The Idea is to apply CNN to Cats&Dogs Image Classification dataset.

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
from zipfile import ZipFile 

In [ ]:
# importing libraries for Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# test_dir="../input/electronic-components/images_test/"
train_dir="../input/electronic-components/images/"

train_dir_LEDS = train_dir + '/LED'
train_dir_transistor = train_dir + '/transistor'
# test_dir_cats = test_dir + '/LED'
# test_dir_dogs = test_dir + '/transistor'

In [ ]:
print('number of LEDs training images - ',len(os.listdir(train_dir_LEDS)))
print('number of transistors training images - ',len(os.listdir(train_dir_transistor)))
# print('number of cats testing images - ',len(os.listdir(test_dir_cats)))
# print('number of dogs testing images - ',len(os.listdir(test_dir_dogs)))

Now we need to convert the RGB images into array of numbers. The requirement can be satisfied by ImageDataGenerator() https://keras.io/preprocessing/image/

In [ ]:
data_generator = ImageDataGenerator(rescale = 1.0/255.0, zoom_range = 0.2)

In [ ]:
batch_size = 32
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                   target_size = (64, 64),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')
# testing_data = data_generator.flow_from_directory(directory = test_dir,
# target_size = (64, 64),
#                                                   batch_size = batch_size,
#                                                   class_mode = 'binary')

In [ ]:
# preparing the layers in the Convolutional Deep Neural Network
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = training_data.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = len(set(training_data.classes)), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
fitted_model = model.fit_generator(training_data,
                        steps_per_epoch = 1000,
                        epochs = 25)

In [ ]:
# plotting accuracy and validation accuracy
accuracy = fitted_model.history['acc']
plt.plot(range(len(accuracy)), accuracy, 'bo', label = 'accuracy')
plt.legend()

In [ ]:
# testing the model
def testing_image(image_directory):
    test_image = image.load_img(image_directory, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(x = test_image)
    print(result)
    if result[0][0]  == 1:
        prediction = 'Dog'
    else:
        prediction = 'Cat'
    return prediction

In [ ]:
print(testing_image(test_dir + '/cats/cat.4003.jpg'))